# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-fb4829ad6a-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Ancient Rome explorative search") 

Consider the following exploratory information need:

> The goal of this workflow is to study the Ancient Rome and some of the main events connected to that historical period. For instance, the emperors, killings, wars and so on.

## Useful URIs for the current workflow
The following data are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q1048`| Julius Caesar       | node |





Also consider that

```
wd:Q1048 ?p ?obj .
```

is the BGP to retrieve all **properties of Julius Caesar**.

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for Ancient Rome

2. Find all the Roman Dictators (return the IRI and the name of the dictator)

3. Identify the BGP for Roman Empire and Roman Republic

4. Return the IRI and the name of all Roman emperors killed by homicide (see "manner of death") and, for each killed emperor, return also the IRI and name of the killer, if present in Wikidata.

5. Return the Roman emperors who killed another Roman emperor (the result set must a list of 4-tuples with killer IRI and label, and killed IRI and label).

6. Find the BGP for war

7. Find the wars (note that "war" and "battle" are not the same thing) participated by the Roman Empire (return IRI and name of the war).

8. Consider all the wars participated by the Roman Empire. For each war return the number of related battles and the opponent (a war is composed of one or more battles) (the result set must be a list of 5-tuples war IRI, label, opponent IRI and label, and #battles).


## Task 1
Identify the BGP for Ancient Rome

In [3]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1048 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3701'), ('name', 'incarnation of')]
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1035'), ('name', 'honorific suffix')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/p

In [4]:
# explore Julius Cesar occupation
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1048 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3409374'), ('name', 'Ancient Roman priest')]
[('obj', 'http://www.wikidata.org/entity/Q99899510'), ('name', 'Ancient Roman historian')]
[('obj', 'http://www.wikidata.org/entity/Q1097498'), ('name', 'ruler')]
[('obj', 'http://www.wikidata.org/entity/Q12859263'), ('name', 'orator')]
[('obj', 'http://www.wikidata.org/entity/Q1402561'), ('name', 'military leader')]
[('obj', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('obj', 'http://www.wikidata.org/entity/Q82955'), ('name', 'politician')]
[('obj', 'http://www.wikidata.org/entity/Q97667506'), ('name', 'Ancient Roman politician')]
[('obj', 'http://www.wikidata.org/entity/Q98103687'), ('name', 'Ancient Roman military personnel')]
[('obj', 'http://www.wikidata.org/entity/Q11774156'), ('name', 'memoirist')]
10


In [5]:
# find Ancient Roman politician properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q97667506 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('name', 'field of this occupation')]
3


In [6]:
# explore field of this occupation
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q97667506 wdt:P425 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q27518'), ('name', 'cursus honorum')]
1


In [7]:
# properties of cursus honorum
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q27518 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('name', 'practiced by')]
[('p', 'http://www.wikidata.org/prop/direct/P3219'), ('name', 'Encyclopædia Universalis ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P4212'), ('name', 'PACTOLS thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5019'), ('name', 'Brockhaus Enzy

In [8]:
# explore country
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q27518 wdt:P17 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1747689'), ('name', 'Ancient Rome')]
1


Final query for this task

In [9]:
# BGP for Ancient Rome
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q27518 wdt:P17 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1747689'), ('name', 'Ancient Rome')]
1


## Task 2
Find all the Roman Dictators (return the IRI and the name of the dictator)

In [10]:
# explore position held
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1048 wdt:P39 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5413489'), ('name', 'curule aedile')]
[('obj', 'http://www.wikidata.org/entity/Q172907'), ('name', 'praetor')]
[('obj', 'http://www.wikidata.org/entity/Q189430'), ('name', 'quaestor')]
[('obj', 'http://www.wikidata.org/entity/Q952844'), ('name', 'pontifex')]
[('obj', 'http://www.wikidata.org/entity/Q1190123'), ('name', 'Roman governor')]
[('obj', 'http://www.wikidata.org/entity/Q20056508'), ('name', 'Ancient Roman senator')]
[('obj', 'http://www.wikidata.org/entity/Q212943'), ('name', 'pontifex maximus')]
[('obj', 'http://www.wikidata.org/entity/Q40779'), ('name', 'Roman consul')]
[('obj', 'http://www.wikidata.org/entity/Q1426927'), ('name', 'Flamen Dialis')]
[('obj', 'http://www.wikidata.org/entity/Q236885'), ('name', 'Roman dictator')]
10


In [11]:
# find all the roman dictators
queryString = """
SELECT DISTINCT ?dictator ?name
WHERE {
   
   ?dictator wdt:P39 wd:Q236885 .
   
   ?dictator sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('dictator', 'http://www.wikidata.org/entity/Q155312'), ('name', 'Gaius Claudius Centho')]
[('dictator', 'http://www.wikidata.org/entity/Q297783'), ('name', 'Appius Claudius Caecus')]
[('dictator', 'http://www.wikidata.org/entity/Q703448'), ('name', 'Marcus Livius Salinator')]
[('dictator', 'http://www.wikidata.org/entity/Q529867'), ('name', 'Titus Lartius')]
[('dictator', 'http://www.wikidata.org/entity/Q15034789'), ('name', 'Manius Valerius Maximus')]
[('dictator', 'http://www.wikidata.org/entity/Q359810'), ('name', 'Lucius Caecilius Metellus')]
[('dictator', 'http://www.wikidata.org/entity/Q929498'), ('name', 'Quintus Caecilius Metellus')]
[('dictator', 'http://www.wikidata.org/entity/Q519186'), ('name', 'Quintus Hortensius')]
[('dictator', 'http://www.wikidata.org/entity/Q637596'), ('name', 'Quintus Servilius Priscus Structus Fidenas')]
[('dictator', 'http://www.wikidata.org/entity/Q1223522'), ('name', 'Caius Poetelius Libo Visolus')]
[('dictator', 'http://www.wikidata.org

Final query for this task

In [12]:
# find all the roman dictators
queryString = """
SELECT DISTINCT ?dictator ?name
WHERE {
   
   ?dictator wdt:P39 wd:Q236885 .
   
   ?dictator sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('dictator', 'http://www.wikidata.org/entity/Q155312'), ('name', 'Gaius Claudius Centho')]
[('dictator', 'http://www.wikidata.org/entity/Q297783'), ('name', 'Appius Claudius Caecus')]
[('dictator', 'http://www.wikidata.org/entity/Q703448'), ('name', 'Marcus Livius Salinator')]
[('dictator', 'http://www.wikidata.org/entity/Q529867'), ('name', 'Titus Lartius')]
[('dictator', 'http://www.wikidata.org/entity/Q15034789'), ('name', 'Manius Valerius Maximus')]
[('dictator', 'http://www.wikidata.org/entity/Q359810'), ('name', 'Lucius Caecilius Metellus')]
[('dictator', 'http://www.wikidata.org/entity/Q929498'), ('name', 'Quintus Caecilius Metellus')]
[('dictator', 'http://www.wikidata.org/entity/Q519186'), ('name', 'Quintus Hortensius')]
[('dictator', 'http://www.wikidata.org/entity/Q637596'), ('name', 'Quintus Servilius Priscus Structus Fidenas')]
[('dictator', 'http://www.wikidata.org/entity/Q1223522'), ('name', 'Caius Poetelius Libo Visolus')]
[('dictator', 'http://www.wikidata.org

## Task 3
Identify the BGP for Roman Empire and Roman Republic

In [13]:
# properties of Ancient Rome
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1747689 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1667'), ('name', 'Getty Thesaurus of Geographic Names ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1792'), ('name', 'category of associated people')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'im

In [17]:
# explore instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1747689 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3024240'), ('name', 'historical country')]
1


In [18]:
# explore has part
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1747689 wdt:P527 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2277'), ('name', 'Roman Empire')]
[('obj', 'http://www.wikidata.org/entity/Q12544'), ('name', 'Byzantine Empire')]
[('obj', 'http://www.wikidata.org/entity/Q17167'), ('name', 'Roman Republic')]
[('obj', 'http://www.wikidata.org/entity/Q201038'), ('name', 'Roman Kingdom')]
4


Final query for this task

In [19]:
# BGP for Roman Empire and Roman Republic
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1747689 wdt:P527 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2277'), ('name', 'Roman Empire')]
[('obj', 'http://www.wikidata.org/entity/Q12544'), ('name', 'Byzantine Empire')]
[('obj', 'http://www.wikidata.org/entity/Q17167'), ('name', 'Roman Republic')]
[('obj', 'http://www.wikidata.org/entity/Q201038'), ('name', 'Roman Kingdom')]
4


## Task 4
Return the IRI and the name of all Roman emperors killed by homicide (see "manner of death") and, for each killed emperor, return also the IRI and name of the killer, if present in Wikidata.

In [20]:
# explore Roman Governor properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1190123 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('name', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
9


In [21]:
# explore has parts of the class
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1190123 wdt:P2670 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q172907'), ('name', 'praetor')]
[('obj', 'http://www.wikidata.org/entity/Q40779'), ('name', 'Roman consul')]
[('obj', 'http://www.wikidata.org/entity/Q499165'), ('name', 'Procurator')]
3


In [22]:
# explore subclass of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1190123 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q20778343'), ('name', 'Roman magistrate')]
1


In [23]:
# explore Roman Empire properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q2277 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('name', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('name', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1464'), ('name', 'category for people born here')]
[('p', 'http://www.wikidata.org/prop/direct/P1465'), ('name', 'category for people who died here')]
[('p', 'http://www.wikid

In [24]:
# explore category of associated people
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q2277 wdt:P1792 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8678282'), ('name', 'Category:People of the Roman Empire')]
1


In [25]:
# explore properties of Category:People of the Roman Empire
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   wd:Q8678282 ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P4224'), ('name', 'category contains')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]
3


In [26]:
# explore category contains
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q8678282 wdt:P4224 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5'), ('name', 'human')]
1


In [27]:
# explore held of state Roman Empire
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q2277 wdt:P1906 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q842606'), ('name', 'Roman emperor')]
1


In [28]:
# find instamces of Roman emperor (wrong)
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q842606 .
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [3]:
# properties of Roman Emperor
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q842606 ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('name', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P1415'), ('name', 'Oxford Dictionary of National Biography ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P373

In [6]:
# explore properties that have Roman emperor as object
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s ?p wd:Q842606 .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('name', 'depicts')]
[('p', 'http://www.wikidata.org/prop/direct/P1906'), ('name', 'office held by head of state')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('name', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P39'), ('name', 'position held')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]
[('p', 'http://www.wikidata.org/prop/direct/P97'), ('name', 'noble title')]
[('p', 'http://www.wikidata.org/prop/direct/P1817'), ('name', 'addressee')]
10


In [7]:
# find roman emperors (with occupation)
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P106 wd:Q842606 .
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q46840'), ('name', 'Diadumenian')]
1


In [8]:
# find roman emperors (with position held)
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P39 wd:Q842606 .
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q46780'), ('name', 'Aurelian')]
[('s', 'http://www.wikidata.org/entity/Q159798'), ('name', 'Honorius')]
[('s', 'http://www.wikidata.org/entity/Q437604'), ('name', 'Julius Saturninus')]
[('s', 'http://www.wikidata.org/entity/Q211396'), ('name', 'Magnus Maximus')]
[('s', 'http://www.wikidata.org/entity/Q550622'), ('name', 'Maximus of Hispania')]
[('s', 'http://www.wikidata.org/entity/Q1777'), ('name', 'Maximinus Thrax')]
[('s', 'http://www.wikidata.org/entity/Q518890'), ('name', 'Gaius Julius Verus Maximus')]
[('s', 'http://www.wikidata.org/entity/Q172471'), ('name', 'Valens')]
[('s', 'http://www.wikidata.org/entity/Q314828'), ('name', 'Vetranio')]
[('s', 'http://www.wikidata.org/entity/Q187489'), ('name', 'Silvanus')]
[('s', 'http://www.wikidata.org/entity/Q1427'), ('name', 'Hadrian')]
[('s', 'http://www.wikidata.org/entity/Q172168'), ('name', 'Galerius')]
[('s', 'http://www.wikidata.org/entity/Q1429'), ('name', 'Antoninus Pius')]
[('s', 'h

In [9]:
# explore Nero properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q1413 ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1187'), ('name', 'Dharma Drum Institute of Liberal Arts person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [12]:
# explore manner of death
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q1413 wdt:P1196 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10737'), ('name', 'suicide')]
1


In [13]:
# explore all manner of death of Roman emperors
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   ?s wdt:P39 wd:Q842606 ;
      wdt:P1196 ?obj .
   ?obj sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10737'), ('name', 'suicide')]
[('obj', 'http://www.wikidata.org/entity/Q149086'), ('name', 'homicide')]
[('obj', 'http://www.wikidata.org/entity/Q171558'), ('name', 'accident')]
[('obj', 'http://www.wikidata.org/entity/Q18663901'), ('name', 'death in battle')]
[('obj', 'http://www.wikidata.org/entity/Q2746270'), ('name', 'forced suicide')]
[('obj', 'http://www.wikidata.org/entity/Q3739104'), ('name', 'natural causes')]
[('obj', 'http://www.wikidata.org/entity/Q8454'), ('name', 'capital punishment')]
7


In [14]:
# properties of homicide 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q149086 ?p ?obj.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1249'), ('name', 'time of earliest written record')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1542'), ('name', 'has effect')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [15]:
# properties of homicide 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s ?p wd:Q149086.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1399'), ('name', 'convicted of')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('name', 'depicts')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P366'), ('name', 'use')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('name', 'field of this occupation')]
[('p', 'http://www.wikidata.org/

In [17]:
# explore properties of a person convicted of homicide
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s wdt:P1399 wd:Q149086 ;
      ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1593'), ('name', 'defender')]
[('p', 'http://www.wikidata.org/prop/direct/P1345'), ('name', 'number of victims of killer')]
[('p', 'http://www.wikidata.org/prop/direct/P4361'), ('name', 'ExecutedToday ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7106'), ('name', 'LezWatch.TV character ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2291'), ('name', 'charted in')]
[('p', 'http://www.wikidata.org/prop/direct/P3902'), ('name', 'had as last meal')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('n

In [4]:
# find what are the properties that have a killer as object
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?s wdt:P1399 wd:Q149086 .
   ?t ?p ?s.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1591'), ('name', 'defendant')]
[('p', 'http://www.wikidata.org/prop/direct/P6241'), ('name', 'collection creator')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P1441'), ('name', 'present in work')]
[('p', 'http://www.wikidata.org/prop/direct/P157'), ('name', 'killed by')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('name', 'depicts')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('name', 'father')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('name', 'spouse')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('name',

In [3]:
# find roman emperors died of homicide
queryString = """
SELECT DISTINCT ?s ?s_name
WHERE {
   ?s wdt:P39 wd:Q842606 ;
      wdt:P1196 wd:Q149086 ;
      sc:name ?s_name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q46780'), ('s_name', 'Aurelian')]
[('s', 'http://www.wikidata.org/entity/Q437604'), ('s_name', 'Julius Saturninus')]
[('s', 'http://www.wikidata.org/entity/Q1777'), ('s_name', 'Maximinus Thrax')]
[('s', 'http://www.wikidata.org/entity/Q518890'), ('s_name', 'Gaius Julius Verus Maximus')]
[('s', 'http://www.wikidata.org/entity/Q1762'), ('s_name', 'Elagabalus')]
[('s', 'http://www.wikidata.org/entity/Q1411'), ('s_name', 'Claudius')]
[('s', 'http://www.wikidata.org/entity/Q1423'), ('s_name', 'Domitian')]
[('s', 'http://www.wikidata.org/entity/Q185538'), ('s_name', 'Constans')]
[('s', 'http://www.wikidata.org/entity/Q46734'), ('s_name', 'Constantine II')]
[('s', 'http://www.wikidata.org/entity/Q103860'), ('s_name', 'Julius Nepos')]
[('s', 'http://www.wikidata.org/entity/Q1830'), ('s_name', 'Decius')]
[('s', 'http://www.wikidata.org/entity/Q104475'), ('s_name', 'Gallienus')]
[('s', 'http://www.wikidata.org/entity/Q189108'), ('s_name', 'Gratian')

In [5]:
# find the killers of the roman emperors, died of homicide
queryString = """
SELECT DISTINCT ?s ?s_name ?obj ?name
WHERE {
   ?s wdt:P39 wd:Q842606 ;
      wdt:P1196 wd:Q149086 ;
      wdt:P157 ?obj ;
      sc:name ?s_name .
   ?obj sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q103860'), ('s_name', 'Julius Nepos'), ('obj', 'http://www.wikidata.org/entity/Q1228819'), ('name', 'Ovida')]
[('s', 'http://www.wikidata.org/entity/Q189108'), ('s_name', 'Gratian'), ('obj', 'http://www.wikidata.org/entity/Q2846204'), ('name', 'Andragathius')]
[('s', 'http://www.wikidata.org/entity/Q1434'), ('s_name', 'Commodus'), ('obj', 'http://www.wikidata.org/entity/Q980693'), ('name', 'Narcissus')]
[('s', 'http://www.wikidata.org/entity/Q183089'), ('s_name', 'Geta'), ('obj', 'http://www.wikidata.org/entity/Q1446'), ('name', 'Caracalla')]
4


In [4]:
# find any property that contains kill and is related to a roman emperor
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   ?emperor wdt:P39 wd:Q842606 ;
      wdt:P1196 wd:Q149086 ;
      ?p ?obj .
   ?o ?p ?emperor .
   ?p sc:name ?name .
   FILTER (regex(?name, "^kill"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P157'), ('name', 'killed by')]
1


In [5]:
# return the emperors killed of homicide and their killer if present
queryString = """
SELECT DISTINCT ?emperor ?e_name ?killer ?k_name
WHERE {
   ?emperor wdt:P39 wd:Q842606 ;
      wdt:P1196 wd:Q149086 ;
      sc:name ?e_name .
   OPTIONAL { ?emperor wdt:P157 ?killer.
              ?killer sc:name ?k_name. }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('emperor', 'http://www.wikidata.org/entity/Q189108'), ('e_name', 'Gratian'), ('killer', 'http://www.wikidata.org/entity/Q2846204'), ('k_name', 'Andragathius')]
[('emperor', 'http://www.wikidata.org/entity/Q183089'), ('e_name', 'Geta'), ('killer', 'http://www.wikidata.org/entity/Q1446'), ('k_name', 'Caracalla')]
[('emperor', 'http://www.wikidata.org/entity/Q103860'), ('e_name', 'Julius Nepos'), ('killer', 'http://www.wikidata.org/entity/Q1228819'), ('k_name', 'Ovida')]
[('emperor', 'http://www.wikidata.org/entity/Q1434'), ('e_name', 'Commodus'), ('killer', 'http://www.wikidata.org/entity/Q980693'), ('k_name', 'Narcissus')]
[('emperor', 'http://www.wikidata.org/entity/Q46780'), ('e_name', 'Aurelian')]
[('emperor', 'http://www.wikidata.org/entity/Q437604'), ('e_name', 'Julius Saturninus')]
[('emperor', 'http://www.wikidata.org/entity/Q1777'), ('e_name', 'Maximinus Thrax')]
[('emperor', 'http://www.wikidata.org/entity/Q518890'), ('e_name', 'Gaius Julius Verus Maximus')]
[('empero

Final query for this task

In [6]:
# return the emperors killed of homicide and their killer if present
queryString = """
SELECT DISTINCT ?emperor ?e_name ?killer ?k_name
WHERE {
   ?emperor wdt:P39 wd:Q842606 ;
      wdt:P1196 wd:Q149086 ;
      sc:name ?e_name .
   OPTIONAL { ?emperor wdt:P157 ?killer.
              ?killer sc:name ?k_name. }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('emperor', 'http://www.wikidata.org/entity/Q189108'), ('e_name', 'Gratian'), ('killer', 'http://www.wikidata.org/entity/Q2846204'), ('k_name', 'Andragathius')]
[('emperor', 'http://www.wikidata.org/entity/Q183089'), ('e_name', 'Geta'), ('killer', 'http://www.wikidata.org/entity/Q1446'), ('k_name', 'Caracalla')]
[('emperor', 'http://www.wikidata.org/entity/Q103860'), ('e_name', 'Julius Nepos'), ('killer', 'http://www.wikidata.org/entity/Q1228819'), ('k_name', 'Ovida')]
[('emperor', 'http://www.wikidata.org/entity/Q1434'), ('e_name', 'Commodus'), ('killer', 'http://www.wikidata.org/entity/Q980693'), ('k_name', 'Narcissus')]
[('emperor', 'http://www.wikidata.org/entity/Q46780'), ('e_name', 'Aurelian')]
[('emperor', 'http://www.wikidata.org/entity/Q437604'), ('e_name', 'Julius Saturninus')]
[('emperor', 'http://www.wikidata.org/entity/Q1777'), ('e_name', 'Maximinus Thrax')]
[('emperor', 'http://www.wikidata.org/entity/Q518890'), ('e_name', 'Gaius Julius Verus Maximus')]
[('empero

## Task 5
Return the Roman emperors who killed another Roman emperor (the result set must a list of 4-tuples with killer IRI and label, and killed IRI and label).

In [8]:
# find the killers of roman emperors that are also roman emperors (only for manner of death = homicide)
queryString = """
SELECT DISTINCT ?killer ?name ?killed ?k_name
WHERE {
   ?killed wdt:P39 wd:Q842606 ;
      wdt:P1196 wd:Q149086 ;
      sc:name ?k_name ; 
      wdt:P157 ?killer.
   ?killer sc:name ?name;
           wdt:P39 wd:Q842606.
           
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('killer', 'http://www.wikidata.org/entity/Q1446'), ('name', 'Caracalla'), ('killed', 'http://www.wikidata.org/entity/Q183089'), ('k_name', 'Geta')]
1


In [9]:
# find the killers of roman emperors that are also roman emperors (not only for homicide)
queryString = """
SELECT DISTINCT ?killer ?name ?killed ?k_name
WHERE {
   ?killed wdt:P39 wd:Q842606 ;
      sc:name ?k_name ; 
      wdt:P157 ?killer.
   ?killer sc:name ?name;
           wdt:P39 wd:Q842606.
           
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('killer', 'http://www.wikidata.org/entity/Q8413'), ('name', 'Constantine the Great'), ('killed', 'http://www.wikidata.org/entity/Q184549'), ('k_name', 'Licinius')]
[('killer', 'http://www.wikidata.org/entity/Q1446'), ('name', 'Caracalla'), ('killed', 'http://www.wikidata.org/entity/Q183089'), ('k_name', 'Geta')]
2


Final query for this task

In [10]:
# find the killers of roman emperors that are also roman emperors
queryString = """
SELECT DISTINCT ?killer ?name ?killed ?k_name
WHERE {
   ?killed wdt:P39 wd:Q842606 ;
      sc:name ?k_name ; 
      wdt:P157 ?killer.
   ?killer sc:name ?name;
           wdt:P39 wd:Q842606.
           
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('killer', 'http://www.wikidata.org/entity/Q8413'), ('name', 'Constantine the Great'), ('killed', 'http://www.wikidata.org/entity/Q184549'), ('k_name', 'Licinius')]
[('killer', 'http://www.wikidata.org/entity/Q1446'), ('name', 'Caracalla'), ('killed', 'http://www.wikidata.org/entity/Q183089'), ('k_name', 'Geta')]
2


## Task 6
Find the BGP for war

In [3]:
# explore Julius Caesar properties (without ID)
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1048 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   FILTER(!regex(?name, "ID$", "i"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3701'), ('name', 'incarnation of')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1035'), ('name', 'honorific suffix')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', '

In [6]:
# explore Julius Caesar conflict
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1048 wdt:P607 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q202161'), ('name', 'Gallic War')]
[('obj', 'http://www.wikidata.org/entity/Q1238338'), ('name', "Caesar's Civil War")]
2


In [7]:
# explore Gallic War properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q202161 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P242'), ('name', 'locator map image')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P421

In [8]:
# explore instance of Gallic War
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q202161 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q198'), ('name', 'war')]
1


Final query for this task

In [9]:
# find BGP for war
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q202161 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q198'), ('name', 'war')]
1


## Task 7
Find the wars (note that "war" and "battle" are not the same thing) participated by the Roman Empire (return IRI and name of the war).

In [10]:
# explore participant of Gallic War
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q202161 wdt:P710 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q22633'), ('name', 'Germanic peoples')]
[('obj', 'http://www.wikidata.org/entity/Q17167'), ('name', 'Roman Republic')]
[('obj', 'http://www.wikidata.org/entity/Q273854'), ('name', 'Gauls')]
[('obj', 'http://www.wikidata.org/entity/Q849967'), ('name', 'Celtic Britons')]
[('obj', 'http://www.wikidata.org/entity/Q190992'), ('name', 'Iberians')]
[('obj', 'http://www.wikidata.org/entity/Q1255605'), ('name', 'Aquitani')]
6


In [11]:
# find wars participated by Roman Empire
queryString = """
SELECT DISTINCT ?war ?name
WHERE {
   
   ?war wdt:P710 wd:Q2277 ;
        wdt:P31 wd:Q198 ;
        sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('war', 'http://www.wikidata.org/entity/Q6254'), ('name', 'Kitos War')]
[('war', 'http://www.wikidata.org/entity/Q500409'), ('name', 'Roman–Persian Wars')]
[('war', 'http://www.wikidata.org/entity/Q202578'), ('name', 'First Jewish-Roman War')]
[('war', 'http://www.wikidata.org/entity/Q1533828'), ('name', 'Roman–Parthian War of 58–63')]
[('war', 'http://www.wikidata.org/entity/Q3651751'), ('name', "Trajan's Parthian campaign")]
[('war', 'http://www.wikidata.org/entity/Q933963'), ('name', 'Marcomannic Wars')]
[('war', 'http://www.wikidata.org/entity/Q3651748'), ('name', 'Parthian war of Caracalla')]
[('war', 'http://www.wikidata.org/entity/Q1790268'), ('name', 'Roman–Parthian War of 161–166')]
[('war', 'http://www.wikidata.org/entity/Q84850006'), ('name', 'Roman–Parthian War of 195–198')]
9


In [15]:
# find wars in which a roman emperor participated
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   ?emperor wdt:P607 ?obj ;
            wdt:P39 wd:Q842606 .
   ?obj wdt:P31 wd:Q198 .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q55612845'), ('name', 'Gothic Wars')]
1


In [3]:
# explore properties that have a war as object
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   ?s ?p ?war .
   ?war wdt:P31 wd:Q198 .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P1534'), ('name', 'end cause')]
[('p', 'http://www.wikidata.org/prop/direct/P2737'), ('name', 'union of')]
[('p', 'http://www.wikidata.org/prop/direct/P805'), ('name', 'statement is subject of')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), (

In [4]:
# explore subjects of part of of a war
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   
   ?s wdt:P361 ?war .
   ?war wdt:P31 wd:Q198 .
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q11883242'), ('name', 'Moro Rebellion')]
[('s', 'http://www.wikidata.org/entity/Q4289228'), ('name', 'Mexico in World War II')]
[('s', 'http://www.wikidata.org/entity/Q7122359'), ('name', 'Pacific Coast Campaign')]
[('s', 'http://www.wikidata.org/entity/Q15871713'), ('name', '2014 pro-Russian unrest in Ukraine')]
[('s', 'http://www.wikidata.org/entity/Q913604'), ('name', 'Battle of Alma')]
[('s', 'http://www.wikidata.org/entity/Q16200674'), ('name', 'Greek Summer Offensive')]
[('s', 'http://www.wikidata.org/entity/Q16204206'), ('name', 'Operation Sirat-e-Mustaqeem')]
[('s', 'http://www.wikidata.org/entity/Q16212881'), ('name', 'Lunenburg Campaign')]
[('s', 'http://www.wikidata.org/entity/Q402312'), ('name', 'Battle of Dettingen')]
[('s', 'http://www.wikidata.org/entity/Q17011705'), ('name', 'Battle of al-Musayfirah')]
[('s', 'http://www.wikidata.org/entity/Q17509974'), ('name', 'Affair at Galaxara Pass')]
[('s', 'http://www.wikidata.org/en

In [7]:
# explore properties of a war
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   ?war wdt:P31 wd:Q198 ;
        ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7847'), ('name', 'DigitaltMuseum ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1120'), ('name', 'number of deaths')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('name', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1382'), ('name', 'partially coincident with')]
[('p', 'http://www.wikidata.org/prop/direct/P1

In [8]:
# explore part of a war
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   ?war wdt:P31 wd:Q198 ;
        wdt:P361 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4783165'), ('name', 'Arab Cold War')]
[('obj', 'http://www.wikidata.org/entity/Q382116'), ('name', 'Caucasian War')]
[('obj', 'http://www.wikidata.org/entity/Q2592334'), ('name', 'Germanic Wars')]
[('obj', 'http://www.wikidata.org/entity/Q611071'), ('name', 'Chadian–Libyan conflict')]
[('obj', 'http://www.wikidata.org/entity/Q1254630'), ('name', 'Spartan hegemony')]
[('obj', 'http://www.wikidata.org/entity/Q1764124'), ('name', 'Third Mithridatic War')]
[('obj', 'http://www.wikidata.org/entity/Q4511505'), ('name', 'Central Asian possessions of the Russian Empire')]
[('obj', 'http://www.wikidata.org/entity/Q234738'), ('name', 'Turkish War of Independence')]
[('obj', 'http://www.wikidata.org/entity/Q16954835'), ('name', 'Nagorno-Karabakh conflict')]
[('obj', 'http://www.wikidata.org/entity/Q636365'), ('name', 'War of the League of Cambrai')]
[('obj', 'http://www.wikidata.org/entity/Q11079793'), ('name', 'Russo-Persian Wars')]
[('obj', 'htt

In [9]:
# explore part of a war filtering on Roman's wars
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   ?war wdt:P31 wd:Q198 ;
        wdt:P361 ?obj .
   ?obj sc:name ?name.
   FILTER (regex(?name, "^Roma") )
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q122173'), ('name', 'Roman mythology')]
[('obj', 'http://www.wikidata.org/entity/Q1192043'), ('name', 'Roman–Parthian Wars')]
[('obj', 'http://www.wikidata.org/entity/Q500409'), ('name', 'Roman–Persian Wars')]
[('obj', 'http://www.wikidata.org/entity/Q2422375'), ('name', 'Roman conquest of Hispania')]
4


In [5]:
# explore Roman Empire participant in property
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
        wd:Q2277 wdt:P1344 ?obj .
        ?obj sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q63146684'), ('name', 'Battles of Bedriacum')]
1


In [3]:
# explore properties that have Roman Empire as object
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
        ?s ?p wd:Q2277 .
        ?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3701'), ('name', 'incarnation of')]
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('name', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('name', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('name', 'operator')]
[('p', 'http://www.wikidata.org/prop/direct/P1376'), ('name', 'capital of')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidat

In [4]:
# explore subjects of part of that have Roman Empire as object
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
        ?s wdt:P361 wd:Q2277 .
        ?s sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q185103'), ('name', 'Roman Britain')]
[('s', 'http://www.wikidata.org/entity/Q2886278'), ('name', 'Later Roman Empire')]
[('s', 'http://www.wikidata.org/entity/Q787204'), ('name', 'High Roman Empire')]
[('s', 'http://www.wikidata.org/entity/Q842606'), ('name', 'Roman emperor')]
[('s', 'http://www.wikidata.org/entity/Q26897'), ('name', 'Gallia Narbonensis')]
[('s', 'http://www.wikidata.org/entity/Q152136'), ('name', 'Germania Inferior')]
[('s', 'http://www.wikidata.org/entity/Q153553'), ('name', 'Germania Superior')]
[('s', 'http://www.wikidata.org/entity/Q3060427'), ('name', 'Euphratensis')]
[('s', 'http://www.wikidata.org/entity/Q3878417'), ('name', 'Noricum')]
[('s', 'http://www.wikidata.org/entity/Q104671836'), ('name', 'Apse Building, Acropolis, Cyrene')]
[('s', 'http://www.wikidata.org/entity/Q104735263'), ('name', 'Budaraj, sanctuary')]
[('s', 'http://www.wikidata.org/entity/Q106485751'), ('name', 'Ash Sharsharah')]
[('s', 'http://ww

In [5]:
# check if there are wars that are part of Roman Empire
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
        ?s wdt:P361 wd:Q2277 ;
           wdt:P31 wd:Q198 ;
           sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [6]:
# explore subjects of set in period that have Roman Empire as object
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
        ?s wdt:P2408 wd:Q2277 .
        ?s sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q30900668'), ('name', 'Tribune of Rome')]
[('s', 'http://www.wikidata.org/entity/Q135096'), ('name', 'I, Claudius')]
[('s', 'http://www.wikidata.org/entity/Q104911831'), ('name', 'Musée Saint-Raymond,Ra 44')]
[('s', 'http://www.wikidata.org/entity/Q135215'), ('name', 'I, Claudius')]
[('s', 'http://www.wikidata.org/entity/Q104671127'), ('name', 'House of Jason Magnus, Cyrene')]
[('s', 'http://www.wikidata.org/entity/Q104678636'), ('name', 'South Necropolis, Cyrene')]
[('s', 'http://www.wikidata.org/entity/Q76828374'), ('name', 'hoard')]
[('s', 'http://www.wikidata.org/entity/Q80751980'), ('name', 'Ave, Cæsar!')]
[('s', 'http://www.wikidata.org/entity/Q603322'), ('name', 'Pompei: The Legend of Vesuvius')]
[('s', 'http://www.wikidata.org/entity/Q82073818'), ('name', 'Caesareum at Cyrene')]
[('s', 'http://www.wikidata.org/entity/Q82073859'), ('name', 'Temple of Aphrodite in Cyrene')]
[('s', 'http://www.wikidata.org/entity/Q106448575'), ('name'

Final query for this task

In [12]:
# find wars participated by Roman Empire
queryString = """
SELECT DISTINCT ?war ?name
WHERE {
   
   ?war wdt:P710 wd:Q2277 ;
        wdt:P31 wd:Q198 ;
        sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('war', 'http://www.wikidata.org/entity/Q6254'), ('name', 'Kitos War')]
[('war', 'http://www.wikidata.org/entity/Q500409'), ('name', 'Roman–Persian Wars')]
[('war', 'http://www.wikidata.org/entity/Q202578'), ('name', 'First Jewish-Roman War')]
[('war', 'http://www.wikidata.org/entity/Q1533828'), ('name', 'Roman–Parthian War of 58–63')]
[('war', 'http://www.wikidata.org/entity/Q3651751'), ('name', "Trajan's Parthian campaign")]
[('war', 'http://www.wikidata.org/entity/Q933963'), ('name', 'Marcomannic Wars')]
[('war', 'http://www.wikidata.org/entity/Q3651748'), ('name', 'Parthian war of Caracalla')]
[('war', 'http://www.wikidata.org/entity/Q1790268'), ('name', 'Roman–Parthian War of 161–166')]
[('war', 'http://www.wikidata.org/entity/Q84850006'), ('name', 'Roman–Parthian War of 195–198')]
9


## Task 8
Consider all the wars participated by the Roman Empire. For each war return the number of related battles and the opponent (a war is composed of one or more battles) (the result set must be a list of 5-tuples war IRI, label, opponent IRI and label, and #battles).

In [13]:
# explore properties that have wars as object
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   ?war wdt:P31 wd:Q198 .
   ?s ?p ?war .
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P1534'), ('name', 'end cause')]
[('p', 'http://www.wikidata.org/prop/direct/P2737'), ('name', 'union of')]
[('p', 'http://www.wikidata.org/prop/direct/P805'), ('name', 'statement is subject of')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), (

In [10]:
# find BGP for battle (using instance of Battle of jake)
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q296754 wdt:P31 ?obj.
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q178561'), ('name', 'battle')]
1


In [11]:
# explore battles that are part of Roman-Persian wars 
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   
   ?s wdt:P361 wd:Q500409 ;
      wdt:P31 wd:Q178561 ;
      sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2665461'), ('name', 'Battle of Barbalissos')]
[('s', 'http://www.wikidata.org/entity/Q2666515'), ('name', 'Siege of Singara')]
[('s', 'http://www.wikidata.org/entity/Q815110'), ('name', 'Siege of Amida')]
[('s', 'http://www.wikidata.org/entity/Q503605'), ('name', 'Battle of Satala')]
[('s', 'http://www.wikidata.org/entity/Q2671354'), ('name', 'Battle of Ctesiphon')]
[('s', 'http://www.wikidata.org/entity/Q16209451'), ('name', 'Battle of Ctesiphon')]
[('s', 'http://www.wikidata.org/entity/Q2237733'), ('name', 'Battle of Sarus')]
[('s', 'http://www.wikidata.org/entity/Q2243653'), ('name', 'Battle of Resaena')]
[('s', 'http://www.wikidata.org/entity/Q1165502'), ('name', 'Battle of Edessa')]
[('s', 'http://www.wikidata.org/entity/Q1583991'), ('name', 'Battle of Misiche')]
[('s', 'http://www.wikidata.org/entity/Q16208506'), ('name', 'Battle of Carrhae')]
11


In [12]:
# explore properties of Battle of Carrhae
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   wd:Q16208506 ?p ?obj .
   ?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P710'), ('name', 'participant')]
6


In [13]:
# find participants of Battle of Carrhae
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q16208506 wdt:P710 ?obj .
   ?obj sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q83891'), ('name', 'Sasanian Empire')]
[('obj', 'http://www.wikidata.org/entity/Q704151'), ('name', 'Arsacid dynasty of Armenia')]
[('obj', 'http://www.wikidata.org/entity/Q17167'), ('name', 'Roman Republic')]
3


In [4]:
# find participants of Battle of Edessa
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q1165502 wdt:P710 ?obj .
   ?obj sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [15]:
# explore properties that have Roman-Persian wars as object
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   ?s ?p wd:Q500409 .
   ?p sc:name ?name.
   FILTER(!regex(?name, "ID$", "i"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P607'), ('name', 'conflict')]
3


In [7]:
# find the wars participated by Roman Empire with opponents and counting the number of related battles
queryString = """
SELECT DISTINCT ?war ?name ?opponent ?o_name (COUNT(?battle) AS ?n_battles)
WHERE {
   
   ?war wdt:P710 wd:Q2277 ;
        wdt:P31 wd:Q198 ;
        sc:name ?name .
   ?battle wdt:P361 ?war ;
           wdt:P31 wd:Q178561 .
   # find opponent
   ?war wdt:P710 ?opponent .
   ?opponent sc:name ?o_name .
   FILTER (?opponent != wd:Q2277)
   
}
GROUP BY ?war ?name ?opponent ?o_name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('war', 'http://www.wikidata.org/entity/Q500409'), ('name', 'Roman–Persian Wars'), ('opponent', 'http://www.wikidata.org/entity/Q12544'), ('o_name', 'Byzantine Empire'), ('n_battles', '11')]
[('war', 'http://www.wikidata.org/entity/Q500409'), ('name', 'Roman–Persian Wars'), ('opponent', 'http://www.wikidata.org/entity/Q17167'), ('o_name', 'Roman Republic'), ('n_battles', '11')]
[('war', 'http://www.wikidata.org/entity/Q500409'), ('name', 'Roman–Persian Wars'), ('opponent', 'http://www.wikidata.org/entity/Q83891'), ('o_name', 'Sasanian Empire'), ('n_battles', '11')]
[('war', 'http://www.wikidata.org/entity/Q202578'), ('name', 'First Jewish-Roman War'), ('opponent', 'http://www.wikidata.org/entity/Q56273996'), ('o_name', 'Judean Free Government'), ('n_battles', '4')]
[('war', 'http://www.wikidata.org/entity/Q933963'), ('name', 'Marcomannic Wars'), ('opponent', 'http://www.wikidata.org/entity/Q837791'), ('o_name', 'Iazyges'), ('n_battles', '1')]
[('war', 'http://www.wikidata.org/

In [9]:
# find the wars participated by Roman Empire with opponents and counting the number of related battles 
# filter out Byzantine Empire and Roman Repubblic
queryString = """
SELECT DISTINCT ?war ?name ?opponent ?o_name (COUNT(?battle) AS ?n_battles)
WHERE {
   
   ?war wdt:P710 wd:Q2277 ;
        wdt:P31 wd:Q198 ;
        sc:name ?name .
   ?battle wdt:P361 ?war ;
           wdt:P31 wd:Q178561 .
   # find opponent
   ?war wdt:P710 ?opponent .
   ?opponent sc:name ?o_name .
   FILTER (?opponent NOT IN (wd:Q2277, wd:Q12544, wd:Q17167) )
   
}
GROUP BY ?war ?name ?opponent ?o_name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('war', 'http://www.wikidata.org/entity/Q500409'), ('name', 'Roman–Persian Wars'), ('opponent', 'http://www.wikidata.org/entity/Q83891'), ('o_name', 'Sasanian Empire'), ('n_battles', '11')]
[('war', 'http://www.wikidata.org/entity/Q202578'), ('name', 'First Jewish-Roman War'), ('opponent', 'http://www.wikidata.org/entity/Q56273996'), ('o_name', 'Judean Free Government'), ('n_battles', '4')]
[('war', 'http://www.wikidata.org/entity/Q933963'), ('name', 'Marcomannic Wars'), ('opponent', 'http://www.wikidata.org/entity/Q837791'), ('o_name', 'Iazyges'), ('n_battles', '1')]
[('war', 'http://www.wikidata.org/entity/Q933963'), ('name', 'Marcomannic Wars'), ('opponent', 'http://www.wikidata.org/entity/Q308885'), ('o_name', 'Quadi'), ('n_battles', '1')]
[('war', 'http://www.wikidata.org/entity/Q933963'), ('name', 'Marcomannic Wars'), ('opponent', 'http://www.wikidata.org/entity/Q209851'), ('o_name', 'Marcomanni'), ('n_battles', '1')]
[('war', 'http://www.wikidata.org/entity/Q500409'), (

Final query for this task

In [3]:
# find the wars participated by Roman Empire with opponents and counting the number of related battles 
# filter out Byzantine Empire and Roman Repubblic
# Some wars may appear multiple times since there could be multiple opponents
queryString = """
SELECT DISTINCT ?war ?name ?opponent ?o_name (COUNT(?battle) AS ?n_battles)
WHERE {
   
   ?war wdt:P710 wd:Q2277 ;
        wdt:P31 wd:Q198 ;
        sc:name ?name .
   ?battle wdt:P361 ?war ;
           wdt:P31 wd:Q178561 .
   # find opponent
   ?war wdt:P710 ?opponent .
   ?opponent sc:name ?o_name .
   FILTER (?opponent NOT IN (wd:Q2277, wd:Q12544, wd:Q17167) )
   
}
GROUP BY ?war ?name ?opponent ?o_name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('war', 'http://www.wikidata.org/entity/Q500409'), ('name', 'Roman–Persian Wars'), ('opponent', 'http://www.wikidata.org/entity/Q83891'), ('o_name', 'Sasanian Empire'), ('n_battles', '11')]
[('war', 'http://www.wikidata.org/entity/Q202578'), ('name', 'First Jewish-Roman War'), ('opponent', 'http://www.wikidata.org/entity/Q56273996'), ('o_name', 'Judean Free Government'), ('n_battles', '4')]
[('war', 'http://www.wikidata.org/entity/Q933963'), ('name', 'Marcomannic Wars'), ('opponent', 'http://www.wikidata.org/entity/Q837791'), ('o_name', 'Iazyges'), ('n_battles', '1')]
[('war', 'http://www.wikidata.org/entity/Q933963'), ('name', 'Marcomannic Wars'), ('opponent', 'http://www.wikidata.org/entity/Q308885'), ('o_name', 'Quadi'), ('n_battles', '1')]
[('war', 'http://www.wikidata.org/entity/Q933963'), ('name', 'Marcomannic Wars'), ('opponent', 'http://www.wikidata.org/entity/Q209851'), ('o_name', 'Marcomanni'), ('n_battles', '1')]
[('war', 'http://www.wikidata.org/entity/Q500409'), (